# Homework 2 - Linear Regression

Jackie Wang

## PART A: Concepts

1. Model Setup

    a.State the simple linear regression (SLR) setup (ISLP Eq. 3.1) and define each term in the equation.

    yi​=β0​+β1​xi​+ϵi

    yi is the response (output)

    Xi is the predictor (input)

    Bo is the intercept

    B1 is the slope

    e is the random error

    b.List and briefly explain the basic assumptions of the SLR model.

    - Linearlity: Assumption of SLR is that the relationship between the predictor x and the response y have a linaer relationship. 
    - Independ Erros: The error e are independent of each other / erros are uncorrelated
    - Constant variance: the errors have the same variance across the values x
    - the errors are normally distributed with mean 0


2. Model fitting. In your own words, explain how the coefficients of the SLR model are estimated (i.e., how we “fit” the model). 

Coefficients are estimating the best intercept and slope so that the resulting line are as close to our features/data points as possible. In SLR we would get this by minimizing the Residual Sum Of Squares (RSS) total squared differnce between the observed values and the predicted values/  y- yhat.


3. Interpretation. In your own words…

a. Explain the difference between the population regression line and the least squares line.

The population line are from real data it represents the true relationship between X and Y in the entire data. It uses unknown parameters. 

Least Square Lines are estimates of the relationship based on the sample data we have it uses the estimated coefficeints parameters. 

b. Define the standard error and explain what it tells us about the estimated parameters.

The standard errors tells us the how close our estimated parameters are to the true parameters. 

c. Define confidence intervals and explain what it tells us about the estimated parameters.

Confidence intervals is a range of value that captures the true values of a population parameters with a certain level of confidence. It tell us how accurate our estimated parameters are.



Applied example. Consider the SLR model: examScore0+1hoursStudied+

a. State the null and alternative hypotheses being tested in this model.

H0 null : Beta = 0 hour studied has no relationship with exam score
HA alternative: Beta does not equal to 0 hours does have a relationship with exam score

b. Interpret each coefficient: write one sentence for each coefficient explaining its meaning in context of the problem.

Interept (3.4): When a student studies 0 hrs their exam score is 3.4 points
HoursStudied (1.5): for each additonal studied the exam score is expected to increase by 1.5 points on avg.

c. Are the coefficients statistically significant? Explain how you came to that conclusion.

yes both p value are less than 0.001 which is smaller than 0.05 threshold. This mean that both coefficeints are statstically signifcant. 

d. Suppose the 95% confidence intervals (CIs) for 0=[1.8,5.0] and for 1=[1.1,1.9]. Write one sentence of interpretation for each CI in context.

0=[1.8,5.0] we are 95% confident that true avg exam score for students who study 0 hrs lies between 1.8 ad 5.0 points. 

1=[1.1,1.9] we are 95% confident that each additonal hr studied increase the true avg exam score by 1.1 and 1.9 points.

e. For your model, you compute RSE=5.2 and R2=0.62. Write one sentence for each metric that interprets it in the context of the problem.

RSE = 5.2 that means the difference between actual score and predicted exam score is about 5.2 points.
R^2 - 0.62 means that 62% of the variation in exam score can be determined by the number of hrs studied.


**Estimated completion time: 1.5 hours**

**Workflow:**
1) **EDA / exploratory checks** (scatterplots with LS lines)
2) **Fit SLR** with `statsmodels.api.OLS` (manual `add_constant`); interpret
3) **Diagnose SLR** (residuals vs. fitted)
4) **Fit MLR** with **backward selection** using `statsmodels.api.OLS` (numeric predictors only)
5) **Predict with `sklearn.linear_model.LinearRegression`** and compute **RMSE**
6) **Compare SLR vs MLR** on the same split

> **Comments marked **# YOUR CODE HERE** must be completed for the code to run

> **Please also make sure you are pasting your plots and answering any questions in the document in the **main homework document**, not in this notebook.

## Part 0 — Setup
### Importing packages

In [1]:
# If running locally and packages are missing, uncomment:
!pip install -q ISLP statsmodels scikit-learn pandas numpy matplotlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 140)

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 832.4/832.4 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 2.7 MB/s eta 0:00:00


### Load the ISLP College dataset

In [ ]:
College = None
err = None
try:
    from ISLP import load_data
    College = load_data('College')
except Exception as e:
    err = e

if College is None:
    try:
        from ISLP.datasets import load_data as load_data_alt
        College = load_data_alt('College')
    except Exception as e2:
        err = (err, e2)

if College is None:
    raise RuntimeError(f"Could not load College dataset. Error details: {err}")

df = College.copy()
df.columns = df.columns.str.replace(r'[^0-9a-zA-Z]+', '_', regex=True)
print(df.shape)
df.head()

## Part 1 — Exploratory data analysis

We will be modeling **`Outstate`** (out-of-state tuition, continuous) as a function of a chosen predictor.  
**Tasks**
- Print a short summary of `Outstate`.
- Show correlations between `Outstate` and several numeric predictors.
- Make **scatterplots with LS lines** for 2–4 top predictors to visually assess near-linearity.

In [ ]:
target = 'Outstate'
assert target in df.columns, f"{target} not in columns; sample: {df.columns.tolist()[:12]}"
print(df[target].describe())

# Candidate numeric predictors (no qualitative variables such as 'Private')
candidates = [c for c in ['Top10perc','Room_Board','PhD','S_F_Ratio','Expend','Grad_Rate','perc_alumni','Terminal'] if c in df.columns]
if not candidates:
    candidates = [c for c in df.select_dtypes(include=[np.number]).columns if c != target][:8]

print("Candidate numeric predictors:", candidates)
corrs = df[[target]+candidates].corr(numeric_only=True)[target].sort_values(ascending=False)
print("Correlations with Outstate:\n", corrs)

### Pick 2–4 predictors for EDA plots

Set `top_nums` to a list of 2–4 column names from `candidates`.

In [ ]:
top_nums = # YOUR CODE HERE

assert isinstance(top_nums, (list,tuple)) and 1 <= len(top_nums) <= 4, "Choose 2–4 predictors"
for col in top_nums:
    assert col in df.columns, f"{col} must be in df"
print("Top predictors for EDA plots:", top_nums)

### Create the scatterplots + LS lines

Loop over `top_nums`. One figure per predictor.

In [ ]:
for x in top_nums:
  # YOUR CODE HERE

## Part 2 — Fit the SLR (statsmodels OLS)

Pick **one** numeric predictor `x1` for SLR (may be one of `top_nums`; e.g., `Top10perc`). Set `x1` and run the assert.

In [ ]:
x1 = # YOUR CODE HERE
assert isinstance(x1, str) and (x1 in df.columns), f"x1 must be in df; consider: {candidates}"
print("x1 =", x1)

### Fit with statsmodels.api.OLS (manual add_constant) and interpret

1. Construct `X = sm.add_constant(x)` where `x = df[[x1]].values` and `y = df[target].values`.
2. Fit: `model = sm.OLS(y, X).fit()` and print `model.summary()`.
3. Compute **training RMSE** using **sklearn**: fit a `LinearRegression()` on the same `x` → predict; compute `rmse_slr = sqrt(MSE)`.
4. Make plots:
   - Scatter of `Outstate` vs `x1` with LS line
   - Residuals vs fitted (from `model`)

In [ ]:
# 1) Design matrix
x = # YOUR CODE HERE
y = # YOUR CODE HERE
X = # YOUR CODE HERE

# 2) Fit statsmodels OLS
model = # YOUR CODE HERE
print(model.summary())

# 3) RMSE via sklearn LinearRegression
lr_slr = LinearRegression().fit(x, y)  # fit_intercept=True by default
yhat = # YOUR CODE HERE
from math import sqrt
rmse_slr = sqrt(mean_squared_error(y, yhat))
print("SLR RMSE (training):", round(rmse_slr, 2))

# 4) Plots
# YOUR CODE HERE

## Part 3 — Fit the MLR with backward selection (statsmodels for p-values)

- Split data 70/30.
- Start with numeric predictors only (exclude qualitative variables).
- Use **statsmodels OLS** (with `sm.add_constant`) to compute p-values for backward selection on the **training** set.

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

# Numeric candidate pool (exclude the target)
numeric_pool = [c for c in ['Top10perc','Room_Board','PhD','S_F_Ratio','Expend','Grad_Rate','perc_alumni','Terminal'] if c in df.columns]
numeric_pool = [c for c in numeric_pool if c != target]
if not numeric_pool:
    numeric_pool = [c for c in df.select_dtypes(include=[np.number]).columns if c != target][:8]

print("Initial numeric candidate terms:", numeric_pool)

### Implement backward selection by p-value

Rules:
- Fit OLS with `sm.OLS(y_train, sm.add_constant(X_train[kept]))`.
- Use `model.pvalues` (drop `'const'`) to find the worst p-value.
- Drop the worst term if p > 0.05; stop otherwise.

In [ ]:
def fit_sm_ols(X_df, y_series):
    X = sm.add_constant(X_df.values, has_constant='add')
    return sm.OLS(y_series.values, X).fit()

def backward_selection_by_p(train_df, response, terms, alpha_out=0.05):
    kept = terms.copy()
    while True:
        X_train = train_df[kept]
        y_train = train_df[response]
        model = fit_sm_ols(X_train, y_train)
        pvals = pd.Series(model.pvalues, index=['const']+kept).drop('const', errors='ignore')

        worst_term = pvals.idxmax()
        worst_p = pvals.max()

        if worst_p > alpha_out and len(kept) > 1:
            kept.remove(worst_term)
        else:
            break

    final_model = fit_sm_ols(train_df[kept], train_df[response])
    return kept, final_model

kept_terms, mlr_model_train = backward_selection_by_p(train_df, target, numeric_pool, alpha_out=0.05)
print("Selected terms:", kept_terms)
print(mlr_model_train.summary())

## Part 4 — Predict with sklearn and report RMSE + visuals

- Fit a `LinearRegression()` on the **training** set using **only** the selected `kept_terms` (no manual constant).
- Compute **training RMSE** and **test RMSE**.
- Plot **Predicted vs Actual** (test) with a 45° line.
- Plot **Residuals vs Fitted** (test), using sklearn predictions.

In [ ]:
from math import sqrt
Xtr = train_df[kept_terms].values
ytr = train_df[target].values
Xte = test_df[kept_terms].values
yte = test_df[target].values
lr_mlr = LinearRegression().fit(Xtr, ytr)
yhat_tr = lr_mlr.predict(Xtr)
yhat_te = lr_mlr.predict(Xte)
rmse_train = sqrt(mean_squared_error(ytr, yhat_tr))
rmse_test = sqrt(mean_squared_error(yte, yhat_te))
print("MLR RMSE (train):", round(rmse_train,2))
print("MLR RMSE (test):", round(rmse_test,2))

plt.figure(); plt.scatter(yte, yhat_te, alpha=0.6)
lims = [min(yte.min(), yhat_te.min()), max(yte.max(), yhat_te.max())]
plt.plot(lims, lims); plt.xlabel("Actual"); plt.ylabel("Predicted"); plt.title("Predicted vs Actual (Test)"); plt.show()

resid_te = yte - yhat_te
plt.figure(); plt.scatter(yhat_te, resid_te, alpha=0.6); plt.axhline(0, linewidth=1)
plt.xlabel("Fitted (Test)"); plt.ylabel("Residuals"); plt.title("Residuals vs Fitted (Test)"); plt.show()